In [304]:
import binascii
import os.path as osp
import numpy as np
import random

In [314]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    '''Convert string to list of binary packets of length 7 bit'''
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    bits = bits.zfill(8 * ((len(bits) + 7) // 8))
    split_bits = [bits[i+1:i+8] for i in range(0, len(bits), 8)]
    return split_bits

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    '''Convert list of binary packets of length 7 to string'''
    bits = '0' + '0'.join(bits)
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    '''Convert int to bytes'''
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

def generate_K(length=1):
    '''Generate random DNA string sequence of `length` symbols long'''
    nucleotides = ['A', 'T', 'G', 'C']
    secure_random = random.SystemRandom()
    K = ''.join([secure_random.choice(nucleotides) for i in range(length)])
    return K

def wat_comp(dna):
    '''Returns Watson-complimentary of DNA string'''
    comp_dict = {'A': 'T', 'T': 'A', 'G': "C", 'C': 'G'}
    
    complimentary = ''
    for i, nucleotide in enumerate(dna):
        complimentary += comp_dict.get(nucleotide, dna[i])
    return complimentary

def pairing_successful(A, B):
    '''Returns True if A and B can be paired with 100% match'''
    if wat_comp(A) == B:
        return True
    else:
        return False
    
def encrypt(K, M):
    '''Encrypt list of binary strings of length 7 bits `M` to DNA using key `K`'''
    C = []         # Список пакетов зашифрованного сообщения
    M = ''.join(M) # Всё сообщение одной строкой
    seq = 0        # номер пакета
    for i, bit in enumerate(M):
        if int(bit):
            key_part = K[len(K)-10 -10*i : len(K) - 10*i] # 10-мер справа длиной 10 от ключа
            packet = wat_comp(key_part) + str(seq)
            C.append(packet)
            seq += 1
    return C

def decrypt(K, C):
    '''Decrypt DNA string `C` to list of binary strings of length 7 `M_bin` using key `K`'''
    M_bin = ''
    K_parts = [K[i:i+10] for i in range(0, len(K), 10)]

    search_idx = len(K_parts)
    for packet in C:
        msg_id = int(packet[10:])
        msg = packet[:10]

        for part in K_parts[search_idx-1::-1]:
            if pairing_successful(part, msg):
                M_bin += '1'
                search_idx -= 1
                break
            else:
                M_bin += '0'
                search_idx -= 1
    split_bits = [M_bin[i:i+7] for i in range(0, len(M_bin), 7)]
    return split_bits

# Пример работы программы
## Зашифровка

Создаём сообщение: 
$$M~=~``DNA``$$
Переводим сообщеине в двоичный вид – каждая буква отдельно конвертируется в строку из 7 бит. На выходе получаем лист из двоичных строк, каждая длиной 7 бит: 
$$M' = Txt2Bin(ASCII(M))$$

In [315]:
m = 'DNA'           # Исходное сообщение
M = text_to_bits(m) # Преобразовано в двоичный вид
print(M)

['1000100', '1001110', '1000001']


Генерируем ключ `К`, как случайную последовательность ДНК-нуклеотидов длиной $(len(M') \cdot 10)$. Так каждому биту из сообщения будет соответствовать случайная последовательность из 10 мер ДНК из ключа.

In [316]:
K = generate_K(len(M)*7*10) # 7 бит в каждой букве
print(K)

ACGACGTGGTCCACCCCCATCGGATATATTTCTCAAGATGAATTCTAAAGTTGAACCATAAATAAACGAATCCAACTGTCGAGGGGATCACGCGAACTAACGCCGAAAGGCTCGTAATGTCGCTTCGTGCTTAGCAGAGCTTGAGGTTACTACCCATGGAATTAAGGATTAACCCGTAGCACTAACGAAGAATCGGTTTTTTGGATGAAT


Зашифровываем сообщение. Идём по сообщению `M` слева-направо. Идём по ключу `K` с конца (справа-налево). Для каждой бинарной `1` из сообщения `M` выбираем кусочек 10-мер ДНК из ключа, находим Ватсон-комплиментарную ДНК (типа биологический XOR), и добавляем к ней номер пакета. Так формируются все пакеты зашифрованных едениц в сообщение `C`.

$$C = E_{KA}(M') = E_{KA}(Bin(ASCII(M)))$$

In [317]:
print(f'M: {M}')
print(f'K: {K}')

C = encrypt(K, M)
print(f'C: {C}')

M: ['1000100', '1001110', '1000001']
K: ACGACGTGGTCCACCCCCATCGGATATATTTCTCAAGATGAATTCTAAAGTTGAACCATAAATAAACGAATCCAACTGTCGAGGGGATCACGCGAACTAACGCCGAAAGGCTCGTAATGTCGCTTCGTGCTTAGCAGAGCTTGAGGTTACTACCCATGGAATTAAGGATTAACCCGTAGCACTAACGAAGAATCGGTTTTTTGGATGAAT
C: ['AACCTACTTA0', 'TAATTCCTAA1', 'AATCGTCTCG2', 'GCGGCTTTCC3', 'GCGCTTGATT4', 'CTCCCCTAGT5', 'TTATTTGCTT6', 'TGCTGCACCA7']


## Дешифровка

Получаем по защищённому каналу ключ `K`. Получаем по открытому каналу зашифрованное в ДНК сообщение `C`. Начинаем обрабатывать *пакеты* из `C`:

1. Отщепляем номер пакета
1. Начинаем сдвигаться справа-налево по ключу с шагом 10, прикладывая 10-мерное сообщение из пакета к ключу. Ищем 100% совпадение (можно спарить ДНК-цепочки, то есть они Ватсон-комплиментарны друг другу)
1. Если цепочки при очередном сдвиге совпали, то добавляем в `M` еденицу
1. Если цепочки при очередном сдвиге не совпали, добавляем в `M` ноль
1. Переводим `M` обратно в текст 

Общая схема дешифровки такова:

$$M_{DEC} = Bin2Txt(D_{KB}(C)) = Bin2Txt(D_{KB}(E_{KA}(M')))$$

$$M_{DEC} = Bin2Txt(D_{KB}(E_{KA}(Txt2Bin(ASCII(M))))$$

In [319]:
# Конкретный пример из статьи (сравнение для поиска ошибок)
# K = 'AGATAGTCATACGTACGACTAACACAGGCATTCACCATGGAACAGCGGTTTCCGTAACATCCTGAGTCCAATAGCGATCCGCTATTCCGTATTTATAAGGGGTCTGATCTTGCATCCGGCATATAGAGCCGATAGGCAGTATGGGGAGTAAGCCGGACCAAGGTCAAATAGGGATTCATCGCCTGATTCCAAAGAATTTGCCGCGCTTCC'
# C = encrypt(K, M)
# print(f'C: {C}')

M_dec_bin = decrypt(K, C)
print(f'M_dec_bin: {M_dec_bin}')

M_dec = text_from_bits(M_dec_bin)
print(f'M_dec    : {M_dec}')

M_dec_bin: ['1000100', '1001110', '1000001']
M_dec    : DNA
